In [19]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [14]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [18]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
#Q1
print(f"Q1: nb columns = {len(df.columns)}")


df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
print(f"Q2: duration STd = {df['duration'].std()}")

before_len = len(df)
df = df[(df.duration >= 1) & (df.duration <= 60)]
after_len = len(df)

print(f"Q3: fraction of the records {after_len/before_len *100} %")

Q1: nb columns = 19
Q2: duration STd = 42.594351241920904
Q3: fraction of the records 98.1220282212598 %


In [25]:
categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [27]:
print(f"Q4: the dimensionality(number of columns): {X_train.shape[1]}")

Q4: the dimensionality(number of columns): 515


In [32]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(f"Q5: RMSE on train: {mean_squared_error(y_train, y_pred, squared=False)}")

Q5: RMSE on train: 7.649261927686161


In [41]:
df2 = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
df2['duration'] = (df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime).apply(lambda td: td.total_seconds() / 60)
df2 = df2[(df2.duration >= 1) & (df2.duration <= 60)]

X_val = dv.transform(df2[categorical].to_dict(orient='records'))


y_val = df2[target].values
y_p_val = lr.predict(X_val)

print(f"Q6: RMSE on val: {mean_squared_error(y_val, y_p_val, squared=False)}")

Q6: RMSE on val: 13.322327465985968
